In [2]:
import os

%matplotlib inline
import string
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import re
import sys
from hashlib import sha1

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
# import tests_hw5
from sklearn import datasets
from sklearn.compose import make_column_transformer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

In [3]:
# !conda list

In [4]:
df = pd.read_csv('UCI_Credit_Card.csv')
print(df.head())

   ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   1    20000.0    2          2         1   24      2      2     -1     -1   
1   2   120000.0    2          2         2   26     -1      2      0      0   
2   3    90000.0    2          2         2   34      0      0      0      0   
3   4    50000.0    2          2         1   37      0      0      0      0   
4   5    50000.0    1          2         1   57     -1      0     -1      0   

   ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0  ...        0.0        0.0        0.0       0.0     689.0       0.0   
1  ...     3272.0     3455.0     3261.0       0.0    1000.0    1000.0   
2  ...    14331.0    14948.0    15549.0    1518.0    1500.0    1000.0   
3  ...    28314.0    28959.0    29547.0    2000.0    2019.0    1200.0   
4  ...    20940.0    19146.0    19131.0    2000.0   36681.0   10000.0   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  default.payment.next.month  
0       0.0       0.0   

In [5]:
X = df.drop('default.payment.next.month', axis=1) 
y = df['default.payment.next.month']              


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=76)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

Training set size: 21000 samples
Test set size: 9000 samples


In [6]:
df.describe()
print(df.dtypes)

ID                              int64
LIMIT_BAL                     float64
SEX                             int64
EDUCATION                       int64
MARRIAGE                        int64
AGE                             int64
PAY_0                           int64
PAY_2                           int64
PAY_3                           int64
PAY_4                           int64
PAY_5                           int64
PAY_6                           int64
BILL_AMT1                     float64
BILL_AMT2                     float64
BILL_AMT3                     float64
BILL_AMT4                     float64
BILL_AMT5                     float64
BILL_AMT6                     float64
PAY_AMT1                      float64
PAY_AMT2                      float64
PAY_AMT3                      float64
PAY_AMT4                      float64
PAY_AMT5                      float64
PAY_AMT6                      float64
default.payment.next.month      int64
dtype: object


In [7]:
from sklearn.impute import SimpleImputer

drop_features = ['ID']
one_hot_features = ['MARRIAGE', 'SEX']
numerical_features = ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 
                      'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 
                      'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
impute_features = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
# ordinal by default, so it's fine
pass_through_features = ['EDUCATION']

preprocessor = make_column_transformer(
    (StandardScaler(), numerical_features),
    (OneHotEncoder(), one_hot_features),
    # Initialize the imputer to replace -2 with 0, and 0 is considered the missing value.
    (SimpleImputer(missing_values=-2, strategy='constant', fill_value=0), impute_features),
    ('drop', drop_features),
    ('passthrough', pass_through_features)
)

preprocessor

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2',
                                  'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5',
                                  'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2',
                                  'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5',
                                  'PAY_AMT6']),
                                ('onehotencoder', OneHotEncoder(),
                                 ['MARRIAGE', 'SEX']),
                                ('simpleimputer',
                                 SimpleImputer(fill_value=0, missing_values=-2,
                                               strategy='constant'),
                                 ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5',
                                  'PAY_6']),
                                ('drop', 'drop', ['ID']),
                                ('passthrough', 'passthrough', ['EDUCATION'])])

In [8]:
baseline_pipe = make_pipeline(preprocessor, DummyClassifier(strategy="prior"))
baseline_pipe.fit(X_train, y_train)
baseline_cross_val_results = pd.DataFrame(
    cross_validate(baseline_pipe, X_train, y_train, return_train_score=True)
)
baseline_cross_val_results

,fit_time,score_time,test_score,train_score
0,0.071116,0.010793,0.778095,0.778155
1,0.036458,0.011054,0.778095,0.778155
2,0.032444,0.009077,0.778095,0.778155
3,0.033183,0.010861,0.778095,0.778155
4,0.031785,0.012078,0.778333,0.778095


In [9]:
linear_pipe = make_pipeline(preprocessor, LogisticRegression(max_iter=1000)) #needed to increase max_iter to avoid ConvergenceWarning
param_grid = {'logisticregression__C': 10.0 ** np.arange(-4, 5)}
gs = GridSearchCV(linear_pipe, param_grid = param_grid, n_jobs=-1, return_train_score=True)
gs.fit(X_train, y_train)
best_alpha = gs.best_params_
print(gs.best_score_, best_alpha)
def mapper(listA, listB):
    return list(map(lambda x, y: "{:.5f} (+/- {:.5f})".format(x, y), listA, listB))
results = {
    'mean fit time': mapper(gs.cv_results_['mean_fit_time'], gs.cv_results_['std_fit_time']),
    'mean score time': mapper(gs.cv_results_['mean_score_time'], gs.cv_results_['std_score_time']),
    'mean validation score': mapper(gs.cv_results_['mean_test_score'], gs.cv_results_['std_test_score']),
    'mean train score': mapper(gs.cv_results_['mean_train_score'], gs.cv_results_['std_train_score']),
}
linear_result = pd.DataFrame(data=results, index=param_grid['logisticregression__C'])
linear_result

0.8126666666666666 {'logisticregression__C': 0.1}


,mean fit time,mean score time,mean validation score,mean train score
0.0001,0.18434 (+/- 0.04143),0.04139 (+/- 0.01020),0.78157 (+/- 0.00169),0.78186 (+/- 0.00046)
0.0010,0.16936 (+/- 0.01790),0.03978 (+/- 0.00219),0.80462 (+/- 0.00494),0.80454 (+/- 0.00126)
0.0100,0.19197 (+/- 0.03899),0.02519 (+/- 0.00292),0.81110 (+/- 0.00454),0.81161 (+/- 0.00127)
0.1000,0.41923 (+/- 0.07554),0.04313 (+/- 0.01705),0.81267 (+/- 0.00431),0.81301 (+/- 0.00129)
1.0000,1.04566 (+/- 0.10864),0.06197 (+/- 0.01807),0.81243 (+/- 0.00445),0.81315 (+/- 0.00129)
10.0000,0.61917 (+/- 0.12175),0.06605 (+/- 0.02602),0.81243 (+/- 0.00442),0.81315 (+/- 0.00130)
100.0000,0.48791 (+/- 0.05513),0.04659 (+/- 0.01648),0.81243 (+/- 0.00442),0.81313 (+/- 0.00128)
1000.0000,0.48578 (+/- 0.08024),0.03381 (+/- 0.00491),0.81243 (+/- 0.00442),0.81315 (+/- 0.00130)
10000.0000,0.44100 (+/- 0.09160),0.03773 (+/- 0.00423),0.81243 (+/- 0.00442),0.81315 (+/- 0.00129)


In [10]:
RFC_pipe = make_pipeline(preprocessor, RandomForestClassifier())
ensemble_pipe = make_pipeline(preprocessor, HistGradientBoostingClassifier())
decision_tree_pipe = make_pipeline(preprocessor, DecisionTreeClassifier())

In [11]:
new_results = {
    'mean fit time': [],
    'mean score time': [],
    'mean validation score': [],
    'mean train score': []
}
def toResults(score):
    for x, y in zip(new_results.keys(),['fit_time', 'score_time', 'test_score', 'train_score']):
        new_results[x].append("{:.5f} (+/- {:.5f})".format(score[y].mean(), score[y].std()))
toResults(cross_validate(decision_tree_pipe, X_train, y_train, return_train_score=True))
toResults(cross_validate(ensemble_pipe, X_train, y_train, return_train_score=True))
toResults(cross_validate(RFC_pipe, X_train, y_train, return_train_score=True))
pd.DataFrame(data=new_results, index=["Decision Tree", "HistGB", "Random Forest"])

,mean fit time,mean score time,mean validation score,mean train score
Decision Tree,0.43402 (+/- 0.06409),0.01044 (+/- 0.00097),0.72076 (+/- 0.00433),0.99962 (+/- 0.00012)
HistGB,1.03494 (+/- 0.16413),0.02380 (+/- 0.00417),0.81810 (+/- 0.00284),0.83768 (+/- 0.00357)
Random Forest,10.94388 (+/- 9.32910),0.12754 (+/- 0.05649),0.81652 (+/- 0.00319),0.99958 (+/- 0.00016)


In [12]:
RFC_param_grid = {'randomforestclassifier__max_depth': [3, 6, 10, 20, 50, 100], 'randomforestclassifier__max_features': [3, 12, 18, 24]}
histGB_param_grid = {'histgradientboostingclassifier__learning_rate': [0.05, 0.1, 0.2, 0.3, 0.5], 'histgradientboostingclassifier__max_depth': [3, 6, 10, 20, 100]}
decision_tree_param_grid = {'decisiontreeclassifier__max_depth': [3, 6, 10, 20, 50, 100], 'decisiontreeclassifier__max_features': [3, 12, 18, 24]}
decision_gs = GridSearchCV(decision_tree_pipe, param_grid = decision_tree_param_grid, n_jobs=-1, return_train_score=True)
decision_gs.fit(X_train, y_train)
histGB_gs = GridSearchCV(ensemble_pipe, param_grid = histGB_param_grid, n_jobs=-1, return_train_score=True)
histGB_gs.fit(X_train, y_train)
RFC_gs = RandomizedSearchCV(RFC_pipe, param_distributions = RFC_param_grid, n_iter=5, n_jobs=-1, return_train_score=True)
RFC_gs.fit(X_train, y_train)
best_results = {
    'mean fit time': [],
    'mean score time': [],
    'mean validation score': [],
    'mean train score': []
}
def toResults(score, index):
    for x, y in zip(best_results.keys(),['fit_time', 'score_time', 'test_score', 'train_score']):
        best_results[x].append("{:.5f} (+/- {:.5f})".format(score['mean_'+y][index], score['std_'+y][index]))
toResults(decision_gs.cv_results_, decision_gs.best_index_)
toResults(histGB_gs.cv_results_, histGB_gs.best_index_)
toResults(RFC_gs.cv_results_, RFC_gs.best_index_)
best_frame = pd.DataFrame(data=best_results, index=['Decision Tree', 'histGB', 'Random Forest'])
best_frame

,mean fit time,mean score time,mean validation score,mean train score
Decision Tree,0.33844 (+/- 0.06604),0.03590 (+/- 0.00909),0.81795 (+/- 0.00395),0.82037 (+/- 0.00107)
histGB,1.49531 (+/- 0.20229),0.05657 (+/- 0.00621),0.82038 (+/- 0.00270),0.83493 (+/- 0.00151)
Random Forest,32.23332 (+/- 10.55804),0.07202 (+/- 0.02924),0.81957 (+/- 0.00439),0.83201 (+/- 0.00100)


In [13]:
learn_rate = histGB_gs.best_params_['histgradientboostingclassifier__learning_rate']
max_depth = histGB_gs.best_params_['histgradientboostingclassifier__max_depth']
best_pipe = make_pipeline(preprocessor, HistGradientBoostingClassifier(learning_rate=learn_rate, max_depth=max_depth))
best_pipe.fit(X_train, y_train)
best_pipe.score(X_test, y_test)

0.8228888888888889

In [ ]:
final_frame = best_frame.copy()
final_frame.loc['Logistic Regression'] = linear_result.loc[gs.best_params_['logisticregression__C']]
final_frame